In [89]:
# Import Dependencies
import pandas as pd

In [90]:
# Import CSV into a Dataframe
df  = pd.read_csv("mempool.csv")
df.head()

,tx_id,fee,weight,parents
0,2e3da8fbc1eaca8ed9b7c2db9e6545d8ccac3c67deadee...,452,1620,NaN
1,79c51c9d4124c5cbb37a85263748dcf44e182dff83561f...,682,1136,6eb38fad135e38a93cb47a15a5f953cbc0563fd84bf1ab...
2,b0ef627c8dc2a706475d33d7712209ec779f7a8302aaea...,226,900,NaN
3,8c25f9be93990b96e8bc363778d6debee6867c7d73cefa...,258,1032,NaN
4,edc82751537be6afcb21c0229d8d20916d8e742a363ab2...,452,1616,NaN


## Defining All Individual Functions

In [91]:
# Sort the Dataframe on the basis of Fee & Weight
# We want to maximise the fee & minimise the weight
# The function takes a dataframe and two features as parameters
def sort_df(df, maxProp, minProp):
    df = df.sort_values([maxProp, minProp], ascending=[False, True]).reset_index(drop=True)
    return df

In [92]:
# Checking if the current Transaction will excede the weight limit
# The function will take the help of 2 global variables - max_weight & current_weight
# The function takes a pandas series as a parameter

def check_weight(x):
    if current_weight + x['weight'] <= max_weight:
        return True
    else:
        return False

In [93]:
# Checking if the transaction already exists in the global list
# The function will take the help of 1 global variable - final_list
# the function takes a transaction as a parameter

def check_in_list(x):
    if str(x) in final_list:
        return True
    else:
        return False

In [94]:
# Checking the parents if the exist in the global list - final_txn and if not, adding them to the list(if eligible) before adding the child 
# The function takes a pandas series and the dataframe as parameters

def check_parent(x):
    parents = str(x[3])
    if parents != "nan":
        parent_list = parents.split(";")
        for i in parent_list:
            if(check_in_list(i)):
                continue
            else:
                txnindex = df[df['tx_id'] == i].index.item()
                k = df.loc[txnindex]
                check_add_txn(k)

In [95]:
# Adding the transaction to the final list - final_list while will be used to generate the .txt file
# The function takes a transaction as a parameter

def add_to_list(x):
    global current_weight
    txnID = x[0]
    weight = x[2]
    current_weight += weight
    final_list.append(txnID)

In [96]:
# All the checks before appending the transaction to the list
# The function takes a transaction as a parameter

def check_add_txn(x):
    if(check_weight(x)):
        if(not check_in_list(x)):
            check_parent(x)
            if(check_weight(x)):
                add_to_list(x)            

In [97]:
# Taking the df, sorting it and iterating over each transaction, checking it and adding it to the final list(if eligible)

def mempoolMain(df):
    sortedDF = sort_df(df, "fee", "weight")
    for i in range(len(sortedDF)):
        txnVar = sortedDF.loc[i]
        check_add_txn(txnVar)

In [110]:
# Creating the text file using the final list

def create_textfile(fin_list):
    file = open("block.txt","a")
    for i in fin_list:
        file.write(str(i) + '\n')
    file.close()

# Main Program

In [111]:
# Declaring the global variables

max_weight = 4000000
current_weight = 0
final_list = []

# feeding the dataframe as "data" imported above to the func - mempoolMain and populating the final_list
data = df
mempoolMain(data)

# Creating the text file using final_list
create_textfile(final_list)